### Assignment1

Remember -> submission using itslearning, both notebook and github repo *deadline on 22.11.2021*

#### Task 1 (3 points):

Regression $\to$ Superconductivity Data Set

The goal here is to predict the critical temperature based on the features extracted.

https://archive.ics.uci.edu/ml/datasets/Superconductivty+Data

* Fit two regression models to predict the critical temperature. Report the score with the default parameters of each model.
* Perform a simple manual optimization for one of the default parameters (at least 5 different values) and plot the new obtained score as a function of the chosen parameter. Plot the coefficient magnitudes for the best model. 

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

# Put the dataset into a dataframe and merge date and timestamp columns into one column
Data = pd.read_csv(r"C:\Users\alext\Desktop\Big Data Analytics\Machine Learning for Predictive Problems\Group assignments\train.csv") 
# Descriptive statistics of the DataFrame
Data.describe().transpose()

In [34]:
import subprocess as sp 
sp.run(['curl',
        '-o',
        'https://archive.ics.uci.edu/ml/machine-learning-databases/00464/superconduct.zip'])
sp.run(['unzip',
        'superconduct.zip'])

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [14]:
Data = Data.select_dtypes(exclude=['object'])
Data.dtypes


number_of_elements         int64
mean_atomic_mass         float64
wtd_mean_atomic_mass     float64
gmean_atomic_mass        float64
wtd_gmean_atomic_mass    float64
                          ...   
range_Valence              int64
wtd_range_Valence        float64
std_Valence              float64
wtd_std_Valence          float64
critical_temp            float64
Length: 82, dtype: object

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = Data.drop("critical_temp", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Data.critical_temp, random_state=42)
#Convert dataframes to numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
lr = LinearRegression().fit(X_train, y_train)

print("Training set score: {:.3f}".format(lr.score(X_train, y_train)))
print("Test set score: {:.3f}".format(lr.score(X_test, y_test)))

Training set score: 0.740
Test set score: 0.728


In [23]:
from sklearn.linear_model import ElasticNet
#Setup an ElasticNet
# define model
EN_model = ElasticNet(max_iter = 10000,tol=0.001)
# Train the model using the training sets
EN_model = EN_model.fit(X_train, y_train)

# Calculate R-squared for both test / train samples
print('Training set score: ' + str(round(EN_model.score(X_train,y_train),3)))
print('Test set score: ' + str(round(EN_model.score(X_test,y_test),3)))

Training set score: 0.719
Test set score: 0.706


In [38]:
alpha = [0.0001,0.001,0.01,0.1,0.5]
rows = ['test','train']
score = pd.DataFrame(columns = rows, index= alpha)
for param in range(5):
    EN_model = ElasticNet(alpha = alpha[param])
    # Train the model using the training sets
    EN_model = EN_model.fit(X_train, y_train)
    # Calculate R-squared for both test / train samples ElasticNet
    score.at[ alpha[param],'train'] = round(EN_model.score(X_train,y_train),3)
    score.at[alpha[param],'test'] = round(EN_model.score(X_test,y_test),3)
score   

C:\Users\alext\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2454299.1832521996, tolerance: 188.00097924087257

C:\Users\alext\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2457613.974107049, tolerance: 188.00097924087257

C:\Users\alext\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2245221.25654074, tolerance: 188.00097924087257

C:\Users\alext\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1278047.1716161992, to

,test,train
0.0001,0.727,0.739
0.0010,0.726,0.738
0.0100,0.722,0.733
0.1000,0.714,0.727
0.5000,0.708,0.722


In [35]:
import plotly.express as px

fig = px.line(score, x=score.index, y=[score.train, score.test], title='Score by parameter')

fig.show()

In [36]:
EN_model = ElasticNet(alpha = 0.0001)
# Train the model using the training sets
EN_model = EN_model.fit(X_train, y_train)
print(EN_model.coef_.round(3))

print(EN_model.intercept_.round(3))

[-4.6280e+00  5.1100e-01 -4.5800e-01 -1.9600e-01  2.3900e-01 -3.8665e+01
  1.0341e+01  2.1400e-01  1.9000e-02 -4.0000e-01 -1.1900e-01  8.9000e-02
  6.6000e-02 -9.1000e-02 -4.5000e-02  4.8297e+01  3.6258e+01  6.8000e-02
  1.5000e-02 -1.9500e-01 -9.0000e-03  1.1430e+00  6.7000e-01 -1.4110e+00
 -3.3600e-01 -1.9322e+01  2.6542e+01  2.4800e-01 -9.5000e-02 -1.1560e+00
  4.0100e-01 -6.0000e-03  0.0000e+00  2.0000e-03  2.0000e-03  1.8523e+01
 -1.4553e+01 -2.0000e-03 -0.0000e+00  6.0000e-03 -1.0000e-03 -2.1100e-01
  6.7000e-01  2.7800e-01 -7.1900e-01  9.8700e-01 -2.0017e+01 -3.9400e-01
 -1.5500e-01  1.3550e+00 -6.3600e-01  9.4200e-01 -1.3490e+00 -7.4800e-01
  1.0140e+00 -1.4717e+01  2.2456e+01 -4.2200e-01  5.1900e-01  1.5300e-01
  2.8500e-01 -5.4000e-02  5.4900e-01 -6.5000e-02 -3.4000e-01  6.6590e+00
  2.6540e+00 -8.9000e-02 -2.2700e-01  2.5800e-01 -1.0000e-02 -8.0200e+00
  4.0910e+00  1.2484e+01 -8.1780e+00  1.7849e+01 -6.0964e+01  5.0940e+00
  5.5700e-01 -1.0880e+00 -1.8971e+01]
-16.986


C:\Users\alext\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2470858.835058255, tolerance: 1880.0097924087256



#### Task 2 (3 points):

Binary classification $\to$ Default of credit card clients Data Set

The goal here is to predict the default payment next month.

https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

* Fit two binary classification models to predict the client's credit card default. Report accuracy with the default parameters of each model.
* Perform a simple manual optimization for one of the default parameters (at least 5 different values) and plot the new obtained accuracy as a function of the chosen parameter.. Plot the feature importance. 

In [41]:
# Put the dataset into a dataframe and merge date and timestamp columns into one column
Data_credit = pd.read_csv(r"C:\Users\alext\Desktop\Big Data Analytics\Machine Learning for Predictive Problems\Group assignments\default of credit card clients.csv",skiprows=[0]) 
# Descriptive statistics of the DataFrame
Data_credit.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
ID,30000.0,15000.500000,8660.398374,1.0,7500.75,15000.5,22500.25,30000.0
LIMIT_BAL,30000.0,167484.322667,129747.661567,10000.0,50000.00,140000.0,240000.00,1000000.0
SEX,30000.0,1.603733,0.489129,1.0,1.00,2.0,2.00,2.0
EDUCATION,30000.0,1.853133,0.790349,0.0,1.00,2.0,2.00,6.0
MARRIAGE,30000.0,1.551867,0.521970,0.0,1.00,2.0,2.00,3.0
AGE,30000.0,35.485500,9.217904,21.0,28.00,34.0,41.00,79.0
PAY_0,30000.0,-0.016700,1.123802,-2.0,-1.00,0.0,0.00,8.0
PAY_2,30000.0,-0.133767,1.197186,-2.0,-1.00,0.0,0.00,8.0
PAY_3,30000.0,-0.166200,1.196868,-2.0,-1.00,0.0,0.00,8.0
PAY_4,30000.0,-0.220667,1.169139,-2.0,-1.00,0.0,0.00,8.0


In [86]:
Data_credit.dtypes

ID                            int64
LIMIT_BAL                     int64
SEX                           int64
EDUCATION                     int64
MARRIAGE                      int64
AGE                           int64
PAY_0                         int64
PAY_2                         int64
PAY_3                         int64
PAY_4                         int64
PAY_5                         int64
PAY_6                         int64
BILL_AMT1                     int64
BILL_AMT2                     int64
BILL_AMT3                     int64
BILL_AMT4                     int64
BILL_AMT5                     int64
BILL_AMT6                     int64
PAY_AMT1                      int64
PAY_AMT2                      int64
PAY_AMT3                      int64
PAY_AMT4                      int64
PAY_AMT5                      int64
PAY_AMT6                      int64
default payment next month    int64
dtype: object

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X = Data_credit.drop("default payment next month", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Data_credit["default payment next month"], random_state=42)
#Convert dataframes to numpy arrays
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()
logr = LogisticRegression(max_iter=10000).fit(X_train, y_train)

print("Training set score: {:.3f}".format(logr.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logr.score(X_test, y_test)))

Training set score: 0.777
Test set score: 0.783


In [43]:
from sklearn.neighbors import KNeighborsClassifier

KNreg = KNeighborsClassifier()
KNreg.fit(X_train,y_train)

print("Training set score: {:.3f}".format(KNreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(KNreg.score(X_test, y_test)))

Training set score: 0.815
Test set score: 0.751


In [44]:
neighbors = [2,7,9,12,20]
rows = ['test','train']
score = pd.DataFrame(columns = rows, index= neighbors)

for param in range(5):
    KNreg = KNeighborsClassifier(n_neighbors = neighbors[param])
    # Train the model using the training sets
    KNreg = KNreg.fit(X_train, y_train)
    # Calculate R-squared for both test / train samples ElasticNet
    score.at[neighbors[param],'train'] = round(KNreg.score(X_train,y_train),3)
    score.at[neighbors[param],'test'] = round(KNreg.score(X_test,y_test),3)
score 

,test,train
2,0.769,0.843
7,0.759,0.804
9,0.767,0.797
12,0.779,0.792
20,0.782,0.785


In [45]:
import plotly.express as px

fig = px.line(score, x=score.index, y=[score.train, score.test], title='Score by parameter')

fig.show()

#### Task 3 (9 points): 

Multiclass & binary classification $\to$ Drug consumption (quantified) Data Set

https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29

Data Set Information:

Database contains records for 1885 respondents. For each respondent 12 attributes are known: Personality measurements which include NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness), BIS-11 (impulsivity), and ImpSS (sensation seeking), level of education, age, gender, country of residence and ethnicity. All input attributes are originally categorical and are quantified. After quantification values of all input features can be considered as real-valued. In addition, participants were questioned concerning their use of 18 legal and illegal drugs (alcohol, amphetamines, amyl nitrite, benzodiazepine, cannabis, chocolate, cocaine, caffeine, crack, ecstasy, heroin, ketamine, legal highs, LSD, methadone, mushrooms, nicotine and volatile substance abuse and one fictitious drug (Semeron) which was introduced to identify over-claimers. For each drug they have to select one of the answers: never used the drug, used it over a decade ago, or in the last decade, year, month, week, or day.

Database contains 18 classification problems. Each of independent label variables contains seven classes: "Never Used", "Used over a Decade Ago", "Used in Last Decade", "Used in Last Year", "Used in Last Month", "Used in Last Week", and "Used in Last Day".

* Fit two multiclass classification models to predict two selected features out of 18. Use numerical values to represent each class. Report accuracy with the default parameters of each model.
* Perform a simple manual optimization for one of the default parameters (at least 5 different values) for one of the previous models. Plot the new obtained accuracy as a function of the chosen parameter. 
* Fit one multiclass classification model for all the rest 16 features. Comment on the accuracy of predicting each feature, for all the seven classes. 
* Run one binary classification model for 3 features out of 18. Test the performance of the model by choosing as:
 - one class (class 0) the variable "Used in Last Decade" and the remaining variables for the other class (class 1).
 - one class (class 0) the variables "Used in Last Decade" and "Used in Last Year" and the remaining variables for the other class (class 1).
 - one class (class 0) the variables "Used in Last Decade", "Used in Last Year", "Used in Last Month" and the remaining variables for the other class (class 1).
 
 Comment your results, and point which selection of classes have better accuracy.

In [160]:
# writing to the file
drug_data =  pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data", header=None)

col_names = ["ID", "Age", "Gender", "Education", "Country", "Ethnicity", "Nscore", "Escore", "Oscore", "Ascore", "Cscore", "Impulsive", "SS", "Alcohol", "Amphet", "Amyl", "Benzos", "Caff", "Cannabis", "Choc", "Coke", "Crack", "Ecstasy", "Heroin", "Ketamine", "Legalh", "LSD", "Meth", "Mushrooms", "Nicotine", "Semer", "VSA"]

drug_data.columns = col_names 

drug_data.dtypes

ID             int64
Age          float64
Gender       float64
Education    float64
Country      float64
Ethnicity    float64
Nscore       float64
Escore       float64
Oscore       float64
Ascore       float64
Cscore       float64
Impulsive    float64
SS           float64
Alcohol       object
Amphet        object
Amyl          object
Benzos        object
Caff          object
Cannabis      object
Choc          object
Coke          object
Crack         object
Ecstasy       object
Heroin        object
Ketamine      object
Legalh        object
LSD           object
Meth          object
Mushrooms     object
Nicotine      object
Semer         object
VSA           object
dtype: object

In [149]:
drug_columns = ['LSD', 'Mushrooms']
drug_new = drug_data
# Encode columns into numeric data
from sklearn.preprocessing import LabelEncoder
for column in drug_columns:
    le = LabelEncoder()
    drug_new[column] = le.fit_transform(drug_new[column])
    

In [150]:
drug_new = drug_new.select_dtypes(exclude=['object'])

In [151]:
X = drug_new.drop(["LSD","Mushrooms"], axis = 1)

X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drug_new["LSD"], random_state=42)
#Convert dataframes to numpy arrays
X_train_drug = X_train_drug.to_numpy()
y_train_drug = y_train_drug.to_numpy()
X_test_drug = X_test_drug.to_numpy()
y_test_drug = y_test_drug.to_numpy()
logr_drug = LogisticRegression(max_iter=10000,multi_class='multinomial').fit(X_train_drug, y_train_drug)

print("Training set score: {:.3f}".format(logr_drug.score(X_train_drug, y_train_drug)))
print("Test set score: {:.3f}".format(logr_drug.score(X_test_drug, y_test_drug)))

Training set score: 0.615
Test set score: 0.578


In [85]:
KNreg = KNeighborsClassifier()
KNreg.fit(X_train_drug,y_train_drug)

print("Training set score: {:.3f}".format(KNreg.score(X_train_drug, y_train_drug)))
print("Test set score: {:.3f}".format(KNreg.score(X_test_drug, y_test_drug)))

Training set score: 0.642
Test set score: 0.536


In [86]:
X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drug_new["Mushrooms"], random_state=42)
#Convert dataframes to numpy arrays
X_train_drug = X_train_drug.to_numpy()
y_train_drug = y_train_drug.to_numpy()
X_test_drug = X_test_drug.to_numpy()
y_test_drug = y_test_drug.to_numpy()
logr_drug = LogisticRegression(max_iter=10000,multi_class='multinomial').fit(X_train_drug, y_train_drug)

print("Training set score: {:.3f}".format(logr_drug.score(X_train_drug, y_train_drug)))
print("Test set score: {:.3f}".format(logr_drug.score(X_test_drug, y_test_drug)))

Training set score: 0.564
Test set score: 0.519


In [87]:
KNreg = KNeighborsClassifier()
KNreg.fit(X_train_drug,y_train_drug)

print("Training set score: {:.3f}".format(KNreg.score(X_train_drug, y_train_drug)))
print("Test set score: {:.3f}".format(KNreg.score(X_test_drug, y_test_drug)))

Training set score: 0.623
Test set score: 0.445


In [88]:
C = [0.0001, 0.001, 0.01, 0.1, 1]
rows = ['test','train']
score = pd.DataFrame(columns = rows, index= l1_ratio)

for param in range(5):
    logr_drug = LogisticRegression(max_iter= 10000, C=C[param])
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples ElasticNet
    score.at[l1_ratio[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[l1_ratio[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 

,test,train
0.0001,0.511,0.524
0.0010,0.508,0.533
0.0100,0.525,0.554
0.1000,0.511,0.569
1.0000,0.519,0.564


In [ ]:
fig = px.line(score, x=score.index, y=[score.train, score.test], title='Score by parameter')

fig.show()

In [152]:
columns_drug = ['Alcohol','Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack',
           'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'Meth','Nicotine', 'Semer', 'VSA']
rows = ['test','train']
score = pd.DataFrame(columns = rows, index= columns_drug)

# Encode columns into numeric data
from sklearn.preprocessing import LabelEncoder
for column in columns_drug:
    le = LabelEncoder()
    drug_data[column] = le.fit_transform(drug_data[column])
    


for param in range(0, len(columns_drug)):
    X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drug_data[columns_drug[param]], random_state=42)
    #Convert dataframes to numpy arrays
    X_train_drug = X_train_drug.to_numpy()
    y_train_drug = y_train_drug.to_numpy()
    X_test_drug = X_test_drug.to_numpy()
    y_test_drug = y_test_drug.to_numpy()
    logr_drug = LogisticRegression(max_iter= 10000, C=0.1)
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples 
    score.at[columns_drug[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[columns_drug[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 
    

,test,train
Alcohol,0.411,0.41
Amphet,0.519,0.518
Amyl,0.695,0.691
Benzos,0.547,0.528
Caff,0.739,0.733
Cannabis,0.405,0.425
Choc,0.419,0.444
Coke,0.523,0.56
Crack,0.881,0.856
Ecstasy,0.54,0.568


In [169]:
drugs_new = drug_data.copy()
columns_drug = ['Crack', 'Heroin', 'Ketamine']

for col in columns_drug:
    drugs_new.loc[(drugs_new[col] == 6) | (drugs_new[col] == 5) | (drugs_new[col] == 4) | (drugs_new[col] == 3) | (drugs_new[col] == 1) | (drugs_new[col] == 0)  ,col] = 1
    drugs_new.loc[(drugs_new[col] == 2) ,col] = 0
   

In [170]:
score = pd.DataFrame(columns = rows, index= columns_drug)
for param in range(0, len(columns_drug)):
    X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drugs_new[columns_drug[param]], random_state=42)
    #Convert dataframes to numpy arrays
    X_train_drug = X_train_drug.to_numpy()
    y_train_drug = y_train_drug.to_numpy()
    X_test_drug = X_test_drug.to_numpy()
    y_test_drug = y_test_drug.to_numpy()
    logr_drug = LogisticRegression(max_iter= 10000, C=0.1)
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples 
    score.at[columns_drug[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[columns_drug[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 
    
    

,test,train
Crack,0.881,0.856
Heroin,0.858,0.849
Ketamine,0.801,0.786


In [171]:
drugs_new = drug_data.copy()
columns_drug = ['Crack', 'Heroin', 'Ketamine']
for col in columns_drug:
    drugs_new.loc[(drugs_new[col] == 6) | (drugs_new[col] == 5) | (drugs_new[col] == 4) | (drugs_new[col] == 1) | (drugs_new[col] == 0)  ,col] = 1
    drugs_new.loc[(drugs_new[col] == 2) | (drugs_new[col] == 3),col] = 0

In [172]:
score = pd.DataFrame(columns = rows, index= columns_drug)
for param in range(0, len(columns_drug)):
    X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drugs_new[columns_drug[param]], random_state=42)
    #Convert dataframes to numpy arrays
    X_train_drug = X_train_drug.to_numpy()
    y_train_drug = y_train_drug.to_numpy()
    X_test_drug = X_test_drug.to_numpy()
    y_test_drug = y_test_drug.to_numpy()
    logr_drug = LogisticRegression(max_iter= 10000, C=0.1)
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples 
    score.at[columns_drug[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[columns_drug[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 

,test,train
Crack,0.881,0.856
Heroin,0.858,0.849
Ketamine,0.801,0.786


In [176]:
drugs_new = drug_data.copy()
columns_drug = ['Crack', 'Heroin', 'Ketamine']
for col in columns_drug:
    drugs_new.loc[(drugs_new[col] == 6) | (drugs_new[col] == 5) | (drugs_new[col] == 1) | (drugs_new[col] == 0)  ,col] = 1
    drugs_new.loc[(drugs_new[col] == 2) | (drugs_new[col] == 3) | (drugs_new[col] == 4),col] = 0
    

In [ ]:
score = pd.DataFrame(columns = rows, index= columns_drug)
for param in range(0, len(columns_drug)):
    X_train_drug, X_test_drug, y_train_drug, y_test_drug = train_test_split(X, drugs_new[columns_drug[param]], random_state=42)
    #Convert dataframes to numpy arrays
    X_train_drug = X_train_drug.to_numpy()
    y_train_drug = y_train_drug.to_numpy()
    X_test_drug = X_test_drug.to_numpy()
    y_test_drug = y_test_drug.to_numpy()
    logr_drug = LogisticRegression(max_iter= 10000, C=0.1)
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples 
    score.at[columns_drug[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[columns_drug[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 

In [ ]:
    # Train the model using the training sets
    logr_drug = logr_drug.fit(X_train_drug, y_train_drug)
    # Calculate R-squared for both test / train samples 
    score.at[columns_drug[param],'train'] = round(logr_drug.score(X_train_drug,y_train_drug),3)
    score.at[columns_drug[param],'test'] = round(logr_drug.score(X_test_drug,y_test_drug),3)
score 

In [3]:
! git commit -am "test_2" 

[main 9a13d25] test_2
 1 file changed, 43 insertions(+), 59 deletions(-)


The file will have its original line endings in your working directory


In [4]:
! git push 

To https://github.com/AlexTouvras/supervised_learning
   942069e..9a13d25  main -> main


In [1]:
! git checkout -b AlexBranch

fatal: A branch named 'AlexBranch' already exists.


In [2]:
! git status

On branch AlexBranch
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Assignment_1.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
! git commit -am "test_branch" 

In [6]:
! git push --set-upstream origin AlexBranch

Branch 'AlexBranch' set up to track remote branch 'AlexBranch' from 'origin'.

remote: 
remote: Create a pull request for 'AlexBranch' on GitHub by visiting:        
remote:      https://github.com/AlexTouvras/supervised_learning/pull/new/AlexBranch        
remote: 
To https://github.com/AlexTouvras/supervised_learning
 * [new branch]      AlexBranch -> AlexBranch
